In [1]:
import sys
import os
import statistics
import math
import json
import pandas as pd
import numpy as np
import random

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

from collections import defaultdict

In [2]:
sys.path.append("..")

In [3]:
from src.data import load_profiles, set_datapath

In [4]:
DATASET = "video_games"
MODE = "all"

In [5]:
line_dict = {}
item_profiles = defaultdict(list)

with open("../../data/video_games/profiles_all.txt", "r") as f:
    for i, line in enumerate(f):
        values = line.strip().split(" ")
        user_id, item_id, ts = int(values[0]), int(values[1]), int(values[2])
        line_dict[(user_id, item_id, ts)] = i
        item_profiles[item_id].append((user_id, ts))

In [7]:
# Random deletion
del_ratio = 0.7
item_subset = {}

for item_id, interactions in item_profiles.items():
    if len(interactions) > 1:
        del_idxs = set(random.sample(range(len(interactions)), int(len(interactions) * del_ratio)))
        item_subset[item_id] = [interaction for i, interaction in enumerate(interactions) if i not in del_idxs]

In [15]:
# Deletion by timestamp
del_ratio = 0.7
item_subset = {}

for item_id, interactions in item_profiles.items():
    if len(interactions) > 1:
        i_sorted = sorted(interactions, key=lambda i: i[1])
        split = int(len(i_sorted) * del_ratio)
        item_subset[item_id] = i_sorted[split:]

In [6]:
# Random select and delete oldest
del_ratio = 0.6
inter_total = sum(len(p) for p in item_profiles.values())
inter_to_del = int(inter_total * del_ratio)
deleted = 0
item_ids = list(item_profiles.keys())

while deleted < inter_to_del:
    item_id = random.randint(0, len(item_ids) - 1)
    if len(item_profiles[item_ids[item_id]]) > 1:
        item_profiles[item_ids[item_id]] = item_profiles[item_ids[item_id]][1:]
        deleted += 1

In [7]:
# Item statistics
avg_i = statistics.mean(len(p) for p in item_profiles.values())
std_i = statistics.stdev(len(p) for p in item_profiles.values())
med_i = statistics.median(len(p) for p in item_profiles.values())
mod_i = statistics.mode(len(p) for p in item_profiles.values())
max_i = max(len(p) for p in item_profiles.values())
min_i = min(len(p) for p in item_profiles.values())

In [53]:
# Item statistics
avg_s = statistics.mean(len(p) for p in item_subset.values())
std_s = statistics.stdev(len(p) for p in item_subset.values())
med_s = statistics.median(len(p) for p in item_subset.values())
mod_s = statistics.mode(len(p) for p in item_subset.values())
max_s = max(len(p) for p in item_subset.values())
min_s = min(len(p) for p in item_subset.values())

In [8]:
print("; ".join(str(x) for x in [avg_i, std_i, med_i, mod_i, max_i, min_i]))

13.839169494353897; 112.54690849263997; 1.0; 1; 6300; 1


In [54]:
print("; ".join(str(x) for x in [avg_s, std_s, med_s, mod_s, max_s, min_s]))

356.9397417503587; 508.9733575186983; 185; 7; 6215; 1


In [22]:
item_subset = item_profiles

In [23]:
profiles = []

for item_id, interactions in item_subset.items():
    for user_id, ts in interactions:
        profiles.append((user_id, item_id, ts))

In [24]:
p_sorted = sorted(profiles, key=lambda p: p[2])

In [25]:
with open("../../data/video_games/profiles_sub_time2.txt", "w") as f:
    for uid, iid, ts in p_sorted:
        f.write(f"{uid} {iid} {ts}\n")

In [26]:
len(p_sorted) / sum(len(p) for p in item_profiles.values())

1.0

In [27]:
len(p_sorted)

746361